In [6]:
import pickle
import pandas as pd
import numpy as np
import re
from bs4 import BeautifulSoup
from kiwipiepy import Kiwi
from typing import Tuple
from tqdm import tqdm

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [7]:
kiwi = Kiwi()

# Helpers

In [8]:
def extract_words_with_target_tags(sentence: str, tags: Tuple[str] = ('NNG', 'NNP')):
    parsed_sentence = kiwi.analyze(sentence)[0][0]
    
    target_tokens = [token.form for token in parsed_sentence if token.tag in tags]
    
    return target_tokens

In [9]:
class UnionFind:
    def __init__(self, n):
        self.parent = [i for i in range(n)]
        self.rank = [0] * n

    def find(self, x):
        if self.parent[x] != x:
            self.parent[x] = self.find(self.parent[x])
        return self.parent[x]

    def union(self, x, y):
        root_x = self.find(x)
        root_y = self.find(y)

        if root_x != root_y:
            if self.rank[root_x] < self.rank[root_y]:
                self.parent[root_x] = root_y
            elif self.rank[root_x] > self.rank[root_y]:
                self.parent[root_y] = root_x
            else:
                self.parent[root_x] = root_y
                self.rank[root_y] += 1

In [29]:
def extract_information(item: dict, target: str):
    text = item.get(target)
    if item.get(target):
        text = text.get('choices')[0].get('message').get('content')

    return text

# Data Load

In [31]:
data = pd.read_pickle('./reports_1_summarize_by_gpt.pkl')
data_w_summarization = {key: value for key, value in data.items() if value.get('company_info_summarized')}

## Preprocess

In [32]:
for key, value in tqdm(data_w_summarization.items()):
    value.update({
        'company_info_summarized': extract_information(value, 'company_info_summarized'),
        'business_info_summarized': extract_information(value, 'business_info_summarized'),
    })

100%|██████████| 786/786 [00:00<00:00, 100595.72it/s]


In [35]:
company_info = [value.get('company_info_summarized') for value in data_w_summarization.values()]
company_names = pd.DataFrame([value.get('company_name (kor)') for value in data_w_summarization.values()], columns=['company_name'])

# Model

## TF-IDF

### company info

In [57]:
THRESHOLD = 0.3

company_info_by_vectorized = TfidfVectorizer().fit_transform(company_info).toarray()
company_info_similarity = np.tril(cosine_similarity(company_info_by_vectorized))
np.fill_diagonal(company_info_similarity, 0)
similar_company_info_index = np.where(company_info_similarity > THRESHOLD)

union_find = UnionFind(len(business_info))
for x, y in zip(*similar_company_info_index):
    union_find.union(x, y)
    
groups = {i: [] for i in range(len(business_info))}
for i, parent in enumerate(union_find.parent):
    groups[parent].append(i)

In [58]:
groups

{0: [0],
 1: [1],
 2: [2],
 3: [3],
 4: [4],
 5: [5],
 6: [6],
 7: [7],
 8: [8],
 9: [9],
 10: [10],
 11: [11, 544, 600, 754],
 12: [12],
 13: [13],
 14: [14],
 15: [15],
 16: [16],
 17: [17],
 18: [18, 19],
 19: [],
 20: [20],
 21: [21],
 22: [22],
 23: [23],
 24: [24],
 25: [25],
 26: [26],
 27: [27],
 28: [28],
 29: [29],
 30: [30],
 31: [31],
 32: [389],
 33: [33],
 34: [34],
 35: [35],
 36: [36],
 37: [37],
 38: [38],
 39: [39],
 40: [40],
 41: [41],
 42: [42],
 43: [43],
 44: [44],
 45: [45],
 46: [46],
 47: [47],
 48: [48],
 49: [49],
 50: [32,
  50,
  67,
  87,
  120,
  145,
  220,
  260,
  267,
  271,
  333,
  354,
  384,
  385,
  388,
  398,
  427,
  432,
  477,
  498,
  505,
  507,
  518,
  538,
  561,
  597,
  601,
  662,
  683,
  693,
  698,
  699,
  748,
  777,
  781],
 51: [51],
 52: [52],
 53: [53],
 54: [54],
 55: [55],
 56: [56],
 57: [57],
 58: [58],
 59: [59],
 60: [60],
 61: [61],
 62: [62],
 63: [63],
 64: [64],
 65: [65],
 66: [66],
 67: [],
 68: [68],
 69: [69],

In [59]:
np.where(np.array([len(value) for value in groups.values()]) > 10)

(array([50]),)

In [60]:
company_names.iloc[groups.get(50)]

,company_name
32,IHQ보통주
50,LG생활건강보통주
67,NI스틸보통주
87,SK케미칼보통주
120,광주신세계보통주
145,남해화학보통주
220,동원산업보통주
260,모나미보통주
267,미래산업보통주
271,미원상사보통주


### business info

In [61]:
data = pd.read_pickle('./reports_1_summarize_by_gpt.pkl')
data_w_summarization = {key: value for key, value in data.items() if value.get('business_info_summarized')}

In [71]:
for key, value in tqdm(data_w_summarization.items()):
    value.update({
        'business_info_summarized': extract_information(value, 'business_info_summarized'),
    })

100%|██████████| 777/777 [00:00<00:00, 212726.78it/s]


In [72]:
business_info = [value.get('business_info_summarized') for value in data_w_summarization.values()]
company_names = pd.DataFrame([value.get('company_name (kor)') for value in data_w_summarization.values()], columns=['company_name'])

In [93]:
THRESHOLD = 0.14

business_info_by_vectorized = TfidfVectorizer().fit_transform(business_info).toarray()
business_info_similarity = np.tril(cosine_similarity(business_info_by_vectorized))
np.fill_diagonal(business_info_similarity, 0)
similar_business_info_index = np.where(business_info_similarity > THRESHOLD)

union_find = UnionFind(len(business_info))
for x, y in zip(*similar_business_info_index):
    union_find.union(x, y)
    
groups = {i: [] for i in range(len(business_info))}
for i, parent in enumerate(union_find.parent):
    groups[parent].append(i)

In [94]:
np.where(np.array([len(value) for value in groups.values()]) > 10)

(array([ 94, 144, 168]),)

In [95]:
company_names.iloc[groups.get(94)]

,company_name
94,TBH글로벌보통주
100,갤럭시아에스엠보통주
190,대현보통주
192,더블유게임즈보통주
246,롯데케미칼보통주
247,롯데하이마트보통주
378,신성통상보통주
382,신세계인터내셔날보통주
449,엔씨소프트보통주
458,영원무역보통주


In [98]:
data_w_summarization.get('084870').get('business_info_summarized')

'사업의 개요를 간략히 요약하자면, 해당 기업은 패션 산업을 중심으로 온라인 및 오프라인 채널을 통해 다양한 브랜드를 운영하고 있습니다. 주요 영업 브랜드로는 비즈니스 캐주얼 브랜드 마인드브릿지, 여성 의류 브랜드 쥬시쥬디, 캐주얼 브랜드 베이직하우스가 있으며, 온라인 전용 브랜드로 베이직하우스를 전환하여 경쟁력을 강화하고 있습니다.세분화된 시장에서 다양한 브랜드와 제품을 제공하고 있으며, 코로나19로 인한 시장 변화에 대응하며 성장과 수익성 제고를 목표로 하고 있습니다.'

In [100]:
data_w_summarization.get('011420').get('business_info_summarized')

'당사인 갤럭시아 에스엠은 스포츠마케팅과 국내 운동기구 유통사업을 주요 내용으로 하는 기업입니다. 스포츠마케팅 부분에서는 다양한 스포츠 단체 및 선수들과의 협업을 통해 스폰서쉽, 머천다이징, 이벤트 등을 진행하며 스포츠 관련 사업을 활발히 추진하고 있습니다. 또한, 갤럭시아스포츠에서는 골프연습장, 휘트니스, 사우나 등 레져사업을 운영하며 스포츠마케팅 분야에서 새로운 시도를 하고 있습니다. 또한, 스포츠 선수 매니지먼트 사업과 이탈리아 명품 운동기구인 테크노짐의 국내총판권 유통사업을 통해 다양한 영역에서 사업을 발전시키고 있습니다.'

In [102]:
data_w_summarization.get('016090').get('business_info_summarized')

'주요 내용 요약:\n- 사업 시작: 1977년 여성복 브랜드로 출발\n- 브랜드: 5개 브랜드 운영 중\n- 온라인 브랜드: 3개 추가 런칭\n- 유통망: 오프라인 509개 매장 보유\n- 온라인 판매: 대현인사이드 온라인몰 및 다수의 온라인 오픈마켓을 통해 판매\n- 매출: 99.0% 내수시장, 1.0% 수출\n- 2022년 1분기 영업성과: 매출액 786억원, 영업이익 70억원, 당기순이익 59억원\n- 재무상태: 자산총계 2,723억원, 부채총계 496억원, 자본총계 2,227억원, 부채비율은 22.3%로 재무안전성 양호\n- 기업경영: 2000년 이후 흑자 경영 지속, 2021년 결산현금배당 1주당 90원(2020년 1주당 40원)'